In [ ]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature = 0.1)

def plus(a,b):
    return a + b

class CalculatorToolArgsSchema(BaseModel):
    a: int = Field(description="The first number")
    b: int = Field(description="The second number")

class CalculatorTool(BaseTool):
    name: Type[str] = "CalculatorTool"
    description: Type[str] = """
        Use this to perform sums of two numbers.
        The first and second arguments should be numbers.
        Only receives two arguments. 
    """
    args_schema : Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b

agent = initialize_agent(
    llm=llm,
    verbose = True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors = True,
    tools=[
        CalculatorTool()
    ]
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

/var/folders/cm/ycdd7_2d2fq3f98kcv14fznw0000gn/T/ipykernel_7186/572369076.py:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature = 0.1)


PydanticUserError: Field 'name' defined on a base class was overridden by a non-annotated attribute. All field definitions, including overrides, require a type annotation.

For further information visit https://errors.pydantic.dev/2.11/u/model-field-overridden